<a href="https://colab.research.google.com/github/juancuevas-ops/analisis-molecular-covid19/blob/main/prediccion_enla_estructura_del_operon_en_las_bacterias_Bacillus_subtilis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio
from Bio import LogisticRegression
xs =    [[-53, -200.78],
          [117, -267.14],
          [57, -163.47],
          [16, -190.30],
          [11, -220.94],
          [85, -193.94],
          [16, -182.71],
          [15, -180.41],
          [-26, -181.73],
          [58, -259.87],
          [126, -414.53],
          [191, -249.57],
          [113, -265.28],
          [145, -312.99],
          [154, -213.83],
          [147, -380.85],
          [93, -291.13]]

     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 2.3MB 8.0MB/s 


# Nueva sección

 El modelo de regresión logística

16.1.1 Antecedentes y propósitos
La regresión logística es un enfoque de aprendizaje supervisado que intenta distinguir las clases K entre sí utilizando una suma ponderada de algunas variables predictoras xi. El modelo de regresión logística se utiliza para calcular las ponderaciones βi de las variables predictoras. En Biopython, el modelo de regresión logística se implementa actualmente solo para dos clases (K = 2); el número de variables predictoras no tiene límite predefinido.

Como ejemplo, tratemos de predecir la estructura del operon en las bacterias. Un operon es un conjunto de genes adyacentes en la misma hebra de ADN que se transcriben en una sola molécula de ARNm. La traducción de la molécula de ARNM único produce las proteínas individuales. Para Bacillus subtilis, cuyos datos vamos a utilizar, el número promedio de genes en un operon es de aproximadamente 2,4.

Como primer paso para entender la regulación genética en las bacterias, necesitamos conocer la estructura del operon. Para aproximadamente el 10% de los genes en Bacillus subtilis,la estructura de operon se conoce a partir de experimentos. Se puede utilizar un método de aprendizaje supervisado para predecir la estructura del operon para el 90% restante de los genes.

Para un enfoque de aprendizaje supervisado, necesitamos elegir algunas variables predictoras xi que se pueden medir fácilmente y están de alguna manera relacionadas con la estructura operon. Una variable predictora podría ser la distancia en pares base entre genes. Los genes adyacentes pertenecientes al mismo operon tienden a estar separados por una distancia relativamente corta, mientras que los genes adyacentes en diferentes operons tienden a tener un espacio más grande entre ellos para permitir secuencias de promotores y terminadores. Otra variable predictora se basa en mediciones de expresión génica. Por definición, los genes pertenecientes al mismo operon tienen perfiles de expresión génica iguales, mientras que se espera que los genes de diferentes operons tengan perfiles de expresión diferentes. En la práctica, los perfiles de expresión medidos de genes en el mismo operon no son del todo idénticos debido a la presencia de errores de medición. Para evaluar la similitud en los perfiles de expresión génica, suponemos que los errores de medición siguen una distribución normal y calculamos la puntuación de probabilidad de registro correspondiente.

Ahora tenemos dos variables predictoras que podemos utilizar para predecir si dos genes adyacentes en la misma hebra de ADN pertenecen al mismo operon:

x1: el número de pares base entre ellos;
x2: su similitud en el perfil de expresión.
En un modelo de regresión logística, utilizamos una suma ponderada de estos dos predictores para calcular una puntuación conjunta S:

S = β0 + β1 x1 + β2 x2.     (16.1)
El modelo de regresión logística nos da los valores adecuados para los parámetros β0, β1, β2 utilizando dos conjuntos de genes de ejemplo:

OP: Genes adyacentes, en la misma hebra de ADN, conocidos por pertenecer al mismo operon;
NOP: Genes adyacentes, en la misma hebra de ADN, conocidos por pertenecer a diferentes operons.
En el modelo de regresión logística, la probabilidad de pertenecer a una clase depende de la puntuación a través de la función logística. Para las dos clases OP y NOP, podemos escribir esto como

     	
Pr(OP| x1, x2)	 =	
 	
exp(β0 + β1 x1 + β2 x2)
1+exp(β0 + β1 x1 + β2 x2)
   
    (16.2)
Pr(NOP| x1, x2)	 =	
 	
1
1+exp(β0 + β1 x1 + β2 x2)
  
    (16.3)
Utilizando un conjunto de pares genéticos para los que se sabe si pertenecen al mismo operon (clase OP) o a diferentes operons (clase NOP), podemos calcular los pesos β0,β1,β2 maximizando la probabilidad de registro correspondiente a las funciones de probabilidad (16.2) y (16.3).

16.1.2 Formación del modelo de regresión logística

Tabla 16.1: Pares genéticos adyacentes conocidos por pertenecer al mismo operon (clase OP) o a diferentes operons (clase NOP). Las distancias intergéneros son negativas si los dos genes se superponen.
Par genético	Distancia intergénero (x1)	Puntuación de expresión génica (x2)	clase
cotJA — cotJB	-53	-200.78	Op
yesK — yesL	117	-267.14	Op
lplA — lplB	57	-163.47	Op
lplB — lplC	16	-190.30	Op
lplC — lplD	11	-220.94	Op
lplD — yetF	85	-193.94	Op
yfmT — yfmS	16	-182.71	Op
yfmF — yfmE	15	-180.41	Op
citS — citT	-26	-181.73	Op
citM — yflN	58	-259.87	Op
yfiI — yfiJ	126	-414.53	Nop
lipB — yfiQ	191	-249.57	Nop
yfiU — yfiV	113	-265.28	Nop
yfhH — yfhI	145	-312.99	Nop
cotY — cotX	154	-213.83	Nop
yjoB — rapA	147	-380.85	Nop
ptsI — splA	93	-291.13	Nop
En el Cuadro 16.1 se enumeran algunos de los pares genéticos bacillus subtilis para los que se conoce la estructura del operon. Vamos a calcular el modelo de regresión logística a partir de estos datos:

In [2]:
xs

[[-53, -200.78],
 [117, -267.14],
 [57, -163.47],
 [16, -190.3],
 [11, -220.94],
 [85, -193.94],
 [16, -182.71],
 [15, -180.41],
 [-26, -181.73],
 [58, -259.87],
 [126, -414.53],
 [191, -249.57],
 [113, -265.28],
 [145, -312.99],
 [154, -213.83],
 [147, -380.85],
 [93, -291.13]]

In [3]:
 ys = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0]


In [4]:
ys

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [5]:
modelo =  LogisticRegression.train(xs, ys)

Aquí, (xs ) y (ys) están los datos de entrenamiento: xs contiene las variables predictoras para cada par de genes y (ys) especifica si el par de genes pertenece al mismo operón ( 1, clase OP) oa diferentes operones ( 0, clase NOP). El modelo de regresión logística resultante se almacena en model, que contiene los pesos β 0 , β 1 y β 2 :

In [6]:
modelo.beta

[8.98302901571447, -0.03596896044485089, 0.021813956629835193]

Tenga en cuenta que β 1 es negativo, ya que los pares de genes con una distancia entre genes más corta tienen una mayor probabilidad de pertenecer al mismo operón (clase OP). Por otro lado, β 2 es positivo, ya que los pares de genes que pertenecen al mismo operón suelen tener una puntuación de similitud más alta de sus perfiles de expresión génica. El parámetro β 0 es positivo debido a la mayor prevalencia de pares de genes de operón que de pares de genes sin operón en los datos de entrenamiento.

La función traintiene dos argumentos opcionales: update_fny typecode. Laupdate_fn puede usar para especificar una función de devolución de llamada, tomando como argumentos el número de iteración y la probabilidad de registro. Con la función de devolución de llamada, podemos, por ejemplo, realizar un seguimiento del progreso del cálculo del modelo (que utiliza una iteración de Newton-Raphson para maximizar la función de probabilidad logarítmica del modelo de regresión logística):

In [7]:
def show_progress (iteración, loglikelihood):
        print ("Iteración:", iteración, "Función de probabilidad logarítmica:",loglikelihood)

In [8]:
modelo = LogisticRegression.train (xs, ys, update_fn = show_progress)

Iteración: <built-in function iter> Función de probabilidad logarítmica: -11.78350206951907
Iteración: <built-in function iter> Función de probabilidad logarítmica: -7.158867676721056
Iteración: <built-in function iter> Función de probabilidad logarítmica: -5.76877209867943
Iteración: <built-in function iter> Función de probabilidad logarítmica: -5.113622943382592
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.748706424325651
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.5002607714604785
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.311277737371034
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.160150433955946
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.035617197847365
Iteración: <built-in function iter> Función de probabilidad logarítmica: -3.93073282192017
Iteración: <built-in function iter> Función de probabilidad logarítmica: -3.84087

El modelo de regresión logística clasifica yxcE , yxcD como pertenecientes al mismo operón (clase OP), mientras que se predice que yxiB , yxiA pertenecen a diferentes operones:

In [9]:
print ("yxcE, yxcD:", LogisticRegression.classify (modelo, [6, -173.143442352])) 

yxcE, yxcD: 1


In [10]:
print ("yxiB, yxiA:", LogisticRegression.classify (modelo, [ 309, -271.005880394])) 

yxiB, yxiA: 0


abla 16.2: Pares de genes adyacentes de estado de operón desconocido.
Par de genes	Distancia intergene x 1	Puntuación de expresión genética x 2
yxcE - yxcD	6	-173.143442352
yxiB - yxiA	309	-271.005880394

que, por cierto, concuerda con la literatura biológica).

Para averiguar qué tan seguros podemos estar en estas predicciones, podemos llamar a la calculatefunción para obtener las probabilidades (ecuaciones ( 16.2 ) y 16.3 ) para las clases OP y NOP. Para yxcE , yxcD encontramos

In [11]:
q, p = LogisticRegression.calculate (modelo, [6, -173.143442352]) 
print ("clase OP: probabilidad =", p, "clase NOP: probabilidad =", q) 

clase OP: probabilidad = 0.9932421635025626 clase NOP: probabilidad = 0.006757836497437442


y para yxiB , yxiA

In [12]:
q, p = LogisticRegression.calculate (modelo, [309, -271.005880394]) 
print ("clase OP: probabilidad =", p, "clase NOP: probabilidad =", q) 

clase OP: probabilidad = 0.00032121125181733257 clase NOP: probabilidad = 0.9996787887481827


Para tener una idea de la precisión de la predicción del modelo de regresión logística, podemos aplicarlo a los datos de entrenamiento:

In [13]:
for i in range (len (ys)): 
        print ("True:", ys [i], "Predicted:", LogisticRegression.classify (modelo, xs [i])) 

True: 1 Predicted: 1
True: 1 Predicted: 0
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0


mostrando que la predicción es correcta para todos menos uno de los pares de genes. Se puede encontrar una estimación más confiable de la precisión de la predicción a partir de un análisis de omisión, en el que el modelo se vuelve a calcular a partir de los datos de entrenamiento después de eliminar el gen que se va a predecir:

In [14]:
for i in range(len(ys)):
        model = LogisticRegression.train(xs[:i]+xs[i+1:], ys[:i]+ys[i+1:])
        print("True:", ys[i], "Predicted:", LogisticRegression.classify(modelo, xs[i]))


True: 1 Predicted: 1
True: 1 Predicted: 0
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0


La iteración se detiene una vez que el aumento de la función log-verosimilitud es menor que 0,01. Si no se alcanza ninguna convergencia después de 500 iteraciones, la función vuelve con un archivo .trainAssertionError

La palabra clave opcional casi siempre se puede omitir. Esta palabra clave permite al usuario elegir el tipo de matriz numérica que se va a utilizar. En particular, para evitar problemas de memoria para problemas muy grandes, puede ser necesario utilizar flotadores de precisión única (Float8, Float16, etc.) en lugar de doble, que se utiliza de forma predeterminada.typecode

16.1.3 Uso del modelo de regresión logística para la clasificación
La clasificación se realiza llamando a la función. Dado un modelo de regresión logística y los valores de x1 y x2 (por ejemplo, para un par genético de estructura operon desconocida), la función devuelve o, correspondiente a la clase OP y a la clase NOP, respectivamente. Por ejemplo, consideremos los pares genéticos yxcE, yxcD y yxiB, yxiA:classifyclassify10

Pares genéticos adyacentes de estado de operon desconocido.
Par genético	Distancia intergénero x1	Puntuación de expresión génica x2
yxcE — yxcD	6	-173.143442352
yxiB — yxiA	309	-271.005880394


l modelo de regresión logística clasifica yxcE, yxcD como perteneciente al mismo operon (clase OP), mientras que yxiB, yxiA se predice que pertenecen a diferentes operons:

In [16]:
print("yxcE, yxcD:", LogisticRegression.classify(model, [6, -173.143442352]))
print("yxiB, yxiA:", LogisticRegression.classify(model, [309, -271.005880394]))


yxcE, yxcD: 1
yxiB, yxiA: 0


In [ ]:
print("yxcE, yxcD:", LogisticRegression.classify(model, [6, -173.143442352]))
print("yxiB, yxiA:", LogisticRegression.classify(model, [309, -271.005880394]))


In [24]:
print("ptsI, splA:", LogisticRegression.classify(model, [93, -2.9841386725875942]))
print("yfmT — yfmS:", LogisticRegression.classify(model, [16, -182.71]))
#hemos creado la prediccion de este par genetico "yfmT — yfmS: la repuesta del sistema es 1  que es == que decir que pertenecen al mismo operon
# con una exactitud de un  98 % y hemos tomado como variables independientes Distancia intergénero (x1)	Puntuación de expresión génica (x2)



ptsI, splA: 1
yfmT — yfmS: 1


In [25]:
q, p = LogisticRegression.calculate (modelo, [16, -182.71]) 
print ("clase OP: probabilidad =", p, "clase NOP: probabilidad =", q) 

clase OP: probabilidad = 0.9881311443932786 clase NOP: probabilidad = 0.011868855606721351


#CONCLUSION


#hemos creado la prediccion de este par genetico "yfmT — yfmS: la repuesta del sistema es 1  que es == que decir que pertenecen al mismo operon
# con una exactitud de un  98 % y hemos tomado como variables independientes Distancia intergénero (x1)	Puntuación de expresión génica (x2).creado y estudiado para ser usado en el bien de personas que puedan padecer enfermedades de tipo genetica. creado por el cientifico de datos juan cuevas vasquez.
nota  operon de bacteria Bacillus subtilis,